In [1]:
import pandas as pd 

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

pd.options.display.max_columns = None

pd.options.display.max_rows = None
 
pd.options.display.float_format = '{:.6f}'.format

from sklearn.model_selection import train_test_split

import statsmodels
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
plt.rcParams['figure.figsize'] = [15,8]

# import 'Pandas' 
import pandas as pd 

# import 'Numpy' 
import numpy as np

# import subpackage of Matplotlib
import matplotlib.pyplot as plt

# import 'Seaborn' 
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = 50
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format

# import train-test split 
from sklearn.model_selection import train_test_split

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import various functions from sklearn
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

# import the functions for visualizing the decision tree
import pydotplus
from IPython.display import Image  

In [2]:
dt = pd.read_csv('data1.csv')

In [3]:
dt.head()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,0,1,0,0,1,0,0,2,0,0,0,0,0,1,2,-1.165523,-1.285566,0
1,1,1,0,0,0,1,0,0,2,0,2,0,0,0,1,0,3,-0.264071,0.060346,0
2,2,1,0,0,0,1,0,0,2,2,0,0,0,0,0,1,3,-0.367189,-1.244781,1
3,3,1,0,0,0,0,1,0,2,0,2,2,0,0,1,0,0,-0.751387,0.508983,0
4,4,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,2,0.193308,-1.244781,1


In [4]:
dt.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
y = dt.Churn
X = dt.drop('Churn',axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.3)

In [8]:
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

X_train (4907, 18)
X_test (2103, 18)
y_train (4907,)
y_test (2103,)


In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
rnd = RandomForestClassifier(random_state=42)
random_model = rnd.fit(X_train,y_train)

#### Performance of train set

In [11]:
y_pred = random_model.predict(X_train)

In [12]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3589
           1       1.00      1.00      1.00      1318

    accuracy                           1.00      4907
   macro avg       1.00      1.00      1.00      4907
weighted avg       1.00      1.00      1.00      4907



#### Performance of test set

In [13]:
y_test_pred = random_model.predict(X_test)

In [14]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1564
           1       0.60      0.47      0.53       539

    accuracy                           0.78      2103
   macro avg       0.71      0.68      0.69      2103
weighted avg       0.77      0.78      0.77      2103



## Inference:

We can obeserve that train accuracy is 100% and test is 78%. Model is overfitting.

## Hyperperameter tuning using GridSearchCV

In [22]:
grid = {'n_estimators':range(10,100,10),'criterion':['gini','entropy'],'max_depth':range(2,25)}

In [23]:
from sklearn.model_selection import GridSearchCV, KFold

In [24]:
kf = KFold(n_splits=5,shuffle=True,random_state=42)

In [25]:
grid_model = GridSearchCV(estimator=rnd,param_grid=grid,cv=kf)

In [26]:
forest_model = grid_model.fit(X_train,y_train)
print('Best parameters for Random Forest are: ',forest_model.best_params_)

Best parameters for Random Forest are:  {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 70}


In [27]:
final_forest = RandomForestClassifier(criterion='gini',max_depth=9,n_estimators=70,random_state=42)

In [28]:
final_model = final_forest.fit(X_train,y_train)

In [29]:
y_pred = final_model.predict(X_test)

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88      1564
           1       0.68      0.51      0.58       539

    accuracy                           0.81      2103
   macro avg       0.76      0.71      0.73      2103
weighted avg       0.80      0.81      0.80      2103



In [31]:
print(metrics.roc_auc_score(y_test, y_pred))

0.7122946016351205
